# 选择合适的index类型
选择index类型并没有一套精准的法则可以依据，需要根据自己的实际情况选取。下面的几个问题可以作为选取index的参考。

## 是否需要精确的结果
如果需要，应该使用“Flat”
只有 IndexFlatL2 能确保返回精确结果。一般将其作为baseline与其他索引方式对比，以便在精度和时间开销之间做权衡。    
不支持add_with_ids，如果需要，可以用“IDMap, Flat”。  
支持GPU。

In [1]:
import faiss

#数据
import numpy as np 
d = 512          #维数
n_data = 2000   
np.random.seed(0) 
data = []
mu = 3
sigma = 0.1
for i in range(n_data):
    data.append(np.random.normal(mu, sigma, d))
data = np.array(data).astype('float32')

#ids, 6位随机数
ids = []
start = 100000
for i in range(data.shape[0]):
    ids.append(start)
    start += 100
ids = np.array(ids)

In [3]:
#不支持add_with_ids
index = faiss.index_factory(d, "Flat")
index.add(data)
dis, ind = index.search(data[:5], 10)
print(ind)

[[   0  798  879  223  981 1401 1458 1174  919   26]
 [   1  981 1524 1639 1949 1472 1162  923  840  300]
 [   2 1886  375 1351  518 1735 1551 1958  390 1695]
 [   3 1459  331  389  655 1943 1483 1723 1672 1859]
 [   4   13  715 1470  608  459  888  850 1080 1654]]


In [4]:
print(dis)

[[0.       8.007045 8.313328 8.53525  8.560175 8.561642 8.624167 8.628234
  8.709978 8.77004 ]
 [0.       8.27809  8.355579 8.42606  8.462017 8.468868 8.487028 8.549963
  8.562824 8.599199]
 [0.       8.152368 8.156569 8.223303 8.276016 8.376871 8.379269 8.406122
  8.418619 8.443283]
 [0.       8.260519 8.336826 8.339298 8.40288  8.46439  8.474661 8.479043
  8.485248 8.526599]
 [0.       8.346273 8.407202 8.462828 8.49723  8.520801 8.597084 8.600386
  8.605133 8.630594]]


In [4]:
index = faiss.index_factory(d, "IDMap, Flat")
index.add_with_ids(data, ids)
dis, ind = index.search(data[:5], 10)
print(ind)   # 返回的结果是我们自己定义的id

[[100000 179800 187900 122300 198100 240100 245800 217400 191900 102600]
 [100100 198100 252400 263900 294900 247200 216200 192300 184000 130000]
 [100200 288600 137500 235100 151800 273500 255100 295800 139000 269500]
 [100300 245900 133100 138900 165500 294300 248300 272300 267200 285900]
 [100400 101300 171500 247000 160800 145900 188800 185000 208000 265400]]


## 关心内存开销
需要注意的是faiss在索引时必须将index读入内存。
### 如果不在意内存占用空间，使用“HNSWx”
如果内存空间很大，数据库很小，HNSW是最好的选择，速度快，精度高，一般4<=x<=64。不支持add_with_ids，不支持移除向量，不需要训练，不支持GPU。

In [5]:
index = faiss.index_factory(d, "HNSW8")
index.add(data)
dis, ind = index.search(data[:5], 10)
print(ind) 

[[   0  798  223   26  281  807 1515 1639 1168 1013]
 [   1  981 1524 1639 1949 1472 1162  923  840  300]
 [   2 1886  375 1351  518 1958  390 1707 1080 1398]
 [   3 1459  331  389  655 1483 1723 1672 1859  650]
 [   4   13  715 1470  608  459  850 1080 1654  665]]


### 如果稍微有点在意，使用“..., Flat“
"..."是聚类操作，聚类之后将每个向量映射到相应的bucket。该索引类型并不会保存压缩之后的数据，而是保存原始数据，所以内存开销与原始数据一致。通过nprobe参数控制速度/精度。  
支持GPU,但是要注意，选用的聚类操作必须也支持。

In [6]:
index = faiss.index_factory(d, "IVF100, Flat")
index.train(data)
index.add(data)
dis, ind = index.search(data[:5], 10)
print(ind)  

[[   0  879  981 1401  919  143    2  807 1515 1393]
 [   1  511 1504  747  422 1911  638  851 1198 1943]
 [   2  879  807  981 1401 1143  733  441 1324 1280]
 [   3  740  155 1337 1578 1181 1743  290  588 1340]
 [   4 1176  256 1186  574 1459  480 1828  942  305]]


### 如果很在意，使用”PCARx,...,SQ8“
如果保存全部原始数据的开销太大，可以用这个索引方式。包含三个部分，  
1.降维  
2.聚类  
3.scalar 量化，每个向量编码为8bit
不支持GPU

In [7]:
index = faiss.index_factory(d, "PCAR16,IVF50,SQ8")  #每个向量降为16维
index.train(data)
index.add(data)
dis, ind = index.search(data[:5], 10)
print(ind)  

[[   0  671  196 1025 1521  724 1014 1706  879 1281]
 [   1 1564 1008  206 1048  657  294 1406  383  700]
 [   2  927 1792 1948  847 1983 1455   19 1503  878]
 [   3 1778 1750  593 1174 1683  572 1852 1298  446]
 [   4 1457  466  557 1604 1951  912  736  804  836]]


### 如果非常非常在意，使用"OPQx_y,...,PQx"
y需要是x的倍数，一般保持y<=d，y<=4*x。
支持GPU。

In [8]:
index = faiss.index_factory(d, "OPQ32_512,IVF50,PQ32")  
index.train(data)
index.add(data)
dis, ind = index.search(data[:5], 10)
print(ind)  

[[   0 1186 1686 1552 1707  125 1563   47 1341 1666]
 [   1 1678  747 1816   41  511  422  748 1179 1632]
 [   2 1507  339 1080 1156 1956  278 1565  936  923]
 [   3   89  548 1337  509 1743  865  731 1578 1862]
 [   4 1545 1293  223  860  976  717  444  794 1457]]


## 数据集的大小
在高效检索的index中，聚类是其中的基础操作，数据量的大小主要影响聚类过程。
### 如果小于1M， 使用"...,IVFx,..."
N是数据集中向量个数，x一般取值[4*sqrt(N),16*sqrt(N)],需要30*x ～ 256*x个向量的数据集去训练。

### 如果在1M-10M，使用"...,IMI2x10,..."
使用k-means将训练集聚类为2^10个类，但是执行过程是在数据集的两半部分独立执行，即聚类中心有2^(2*10)个。
### 如果在10M-100M，使用"...,IMI2x12,..."